In [2]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, RegressorMixin
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math 
import itertools

In [3]:
dataset=pd.read_csv("iris-setosa.csv")

X = dataset.columns[1:3]
X = dataset[X]
y = dataset.columns[0]
y = dataset[y]

X = np.array(X)
y = np.array(y)
X=X.astype(float)
y=y.astype(float)
y=np.where(y==0,-1,y)

In [4]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma):
    # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

In [8]:
from cvxopt import matrix
class HYP_SVM(BaseEstimator, RegressorMixin):
    
    def __init__(self, kernel=gaussian_kernel, sigma=None, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
        self.sigma = sigma
        
    #geometric mean
    def gm(self, y_predict, y_test):
        print(y_predict)
        test_min=0
        test_max=0
        pred_min=0
        pred_max=0
        y_test=np.asarray(y_test)
        for i in range(0,len(y_test)):
            if(y_test[i]==1):
                test_min=test_min+1
            else:
                test_max=test_max+1
        #print("y_test min",test_min)       
        #print("y_test max",test_max)
        for i in range(0,len(y_predict)):
            if(y_predict[i]==1 and y_predict[i]==y_test[i]):
                pred_min=pred_min+1
            elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
                pred_max=pred_max+1
        #print("y_pred min",pred_min)       
        #print("y_pred max",pred_max)
        se=pred_min/test_min
        sp=pred_max/test_max
        #print(se,sp)
        gm=math.sqrt(se*sp)
        print("GM",gm) 
        return gm
        
    def score(self, X, y):
        y_predict=self.predict(X)
        gm=self.gm(y_predict, y)
        correct = np.sum(y_predict == y)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        mse=mean_squared_error(y, y_predict)
        print("Accuracy",correct/len(y_predict))
        print("Errore quadratico medio: ", mse)
        return gm
            
          
    def get_params(self, deep=True):
        return {"C": self.C, "sigma": self.sigma}        
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def m_func(self, X_train, y):
        n_samples, n_features = X_train.shape 
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        X_train=np.asarray(X_train)
        K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K1[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        #print(K1.shape)
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a_org=a
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        self.sv_yorg=y
        self.kernel = gaussian_kernel
        X_train=np.asarray(X_train)
        b = 0
        for n in range(len(self.a)):
            b += self.sv_y[n]
            b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        b /= len(self.a)
       # print(self.a_org[1])
        #print(self.a_org.shape,self.sv_yorg.shape,K.shape)
        w_phi=0
        total=0
        for n in range(len(self.a_org)):
            w_phi = self.a_org[n] * self.sv_yorg[n] * K1[n] 
        self.d_hyp=np.zeros(n_samples)
        for n in range(len(self.a_org)):
            self.d_hyp += self.sv_yorg[n]*(w_phi+b)
        func=np.zeros((n_samples))
        func=np.asarray(func)
        typ=1
        if(typ==1):
            for i in range(n_samples):
                func[i]=1-(self.d_hyp[i]/(np.amax(self.d_hyp[i])+0.000001))
        beta=0.8
        if(typ==2):
            for i in range(n_samples):
                func[i]=2/(1+beta*self.d_hyp[i])
        r_max=103/4074
        r_min=1
        self.m=func[0:115]*r_min
        #print(self.m.shape)
        self.m=np.append(self.m,func[115:5473]*r_max)
        #print(self.m.shape)
        
 ##############################################################################

    #prendeva come argomento anche x_test, l'ho tolto, ho aggiunto K
    def fit(self, X_train, y):
        
        self.m_func(X_train, y)
        self.kernel = gaussian_kernel
        n_samples, n_features = X_train.shape 

        # Gram matrix

        #print(self.K.shape)

        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        for i in range(n_samples):
            sv=np.logical_or(self.a_org <self.m, self.a_org > 1e-5)
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        #print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        self.b /= len(self.a)
        #print(self.b)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else :
            self.w = None
            
        return self    
        
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv, self.sigma)
                    #print(gaussian_kernel(X[i], sv, self.sigma)>0)
                y_predict[i] = s
                print(y_predict[i]+self.b)
            return y_predict + self.b

    def predict(self, X):
        #return np.sign(self.project(X))
        return self.project(X)

In [6]:
C_vals = [1e-2, 1e-1, 1, 1e+1, 1e+2, 1e+3, 1e+4]
sigma = [9e-2, 9e-1, 9, 9e+1, 9e+2, 9e+3, 9e+4]
#kernels = ["linear_kernel", "polynomial_kernel", "gaussian_kernel"]
parameters = {'C': C_vals, 'sigma': sigma}

model = HYP_SVM(C=100, kernel=gaussian_kernel, sigma=0.9)

xTrain, xTest, yTrain, yTest = train_test_split(X, y)

clf = GridSearchCV(model, parameters, cv=5)

In [9]:
grid_result = clf.fit(X=xTrain, y=yTrain)


     pcost       dcost       gap    pres   dres
 0: -1.5393e+01 -6.2482e+00  4e+02  2e+01  2e-16
 1: -5.3370e+00 -2.2042e+00  3e+01  2e+00  4e-16
 2: -6.2116e-01 -1.6625e+00  1e+00  2e-02  2e-15
 3: -5.9082e-01 -6.3879e-01  5e-02  2e-17  4e-16
 4: -5.9464e-01 -5.9854e-01  4e-03  2e-17  3e-16
 5: -5.9521e-01 -5.9565e-01  4e-04  1e-17  3e-16
 6: -5.9528e-01 -5.9533e-01  5e-05  2e-17  3e-16
 7: -5.9529e-01 -5.9529e-01  4e-06  2e-17  3e-16
 8: -5.9529e-01 -5.9529e-01  9e-08  2e-17  3e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.5393e+01 -6.2482e+00  4e+02  2e+01  2e-16
 1: -5.3370e+00 -2.2042e+00  3e+01  2e+00  4e-16
 2: -6.2116e-01 -1.6625e+00  1e+00  2e-02  2e-15
 3: -5.9082e-01 -6.3879e-01  5e-02  2e-17  4e-16
 4: -5.9464e-01 -5.9854e-01  4e-03  2e-17  3e-16
 5: -5.9521e-01 -5.9565e-01  4e-04  1e-17  3e-16
 6: -5.9528e-01 -5.9533e-01  5e-05  2e-17  3e-16
 7: -5.9529e-01 -5.9529e-01  4e-06  2e-17  3e-16
 8: -5.9529e-01 -5.9529e-01  9e-08  2e-17  3e-1

     pcost       dcost       gap    pres   dres
 0: -6.3172e+00 -3.4741e+00  4e+02  2e+01  4e-16
 1: -2.5398e+00 -2.5915e+00  2e+01  1e+00  4e-16
 2: -7.2407e-01 -1.7259e+00  2e+00  7e-02  2e-15
 3: -5.5265e-01 -8.5338e-01  3e-01  2e-17  1e-15
 4: -5.6449e-01 -6.1504e-01  5e-02  1e-17  5e-16
 5: -5.7234e-01 -5.8171e-01  9e-03  3e-17  2e-16
 6: -5.7529e-01 -5.7772e-01  2e-03  2e-17  3e-16
 7: -5.7588e-01 -5.7691e-01  1e-03  2e-17  3e-16
 8: -5.7616e-01 -5.7658e-01  4e-04  2e-17  2e-16
 9: -5.7633e-01 -5.7640e-01  7e-05  2e-17  3e-16
10: -5.7636e-01 -5.7636e-01  3e-06  2e-17  4e-16
11: -5.7636e-01 -5.7636e-01  3e-08  2e-17  3e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -6.3172e+00 -3.4741e+00  4e+02  2e+01  4e-16
 1: -2.5398e+00 -2.5915e+00  2e+01  1e+00  4e-16
 2: -7.2407e-01 -1.7259e+00  2e+00  7e-02  2e-15
 3: -5.5265e-01 -8.5338e-01  3e-01  2e-17  1e-15
 4: -5.6449e-01 -6.1504e-01  5e-02  1e-17  5e-16
 5: -5.7234e-01 -5.8171e-01  9e-03  3e-17  2e-1

     pcost       dcost       gap    pres   dres
 0: -3.0789e+01 -4.3228e+00  4e+02  2e+01  1e-15
 1: -2.3270e+00 -1.8568e+00  1e+01  6e-01  9e-16
 2: -6.4350e-01 -1.5900e+00  1e+00  1e-02  9e-16
 3: -5.9810e-01 -6.3579e-01  4e-02  1e-04  2e-15
 4: -5.9868e-01 -6.0456e-01  6e-03  2e-05  5e-16
 5: -5.9899e-01 -6.0045e-01  1e-03  2e-06  2e-16
 6: -5.9935e-01 -5.9961e-01  3e-04  3e-07  3e-16
 7: -5.9945e-01 -5.9953e-01  8e-05  8e-08  3e-16
 8: -5.9947e-01 -5.9951e-01  4e-05  3e-08  4e-16
 9: -5.9948e-01 -5.9950e-01  2e-05  2e-08  3e-16
10: -5.9949e-01 -5.9949e-01  5e-06  2e-09  3e-16
11: -5.9949e-01 -5.9949e-01  1e-07  4e-11  3e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.0789e+01 -4.3228e+00  4e+02  2e+01  1e-15
 1: -2.3270e+00 -1.8568e+00  1e+01  6e-01  9e-16
 2: -6.4350e-01 -1.5900e+00  1e+00  1e-02  9e-16
 3: -5.9810e-01 -6.3579e-01  4e-02  1e-04  2e-15
 4: -5.9868e-01 -6.0456e-01  6e-03  2e-05  5e-16
 5: -5.9899e-01 -6.0045e-01  1e-03  2e-06  2e-1

     pcost       dcost       gap    pres   dres
 0: -4.0033e+01 -1.9239e+00  5e+02  2e+01  1e-15
 1: -9.8792e-01 -1.8697e+00  6e+00  2e-01  1e-15
 2: -5.9123e-01 -1.3063e+00  7e-01  4e-17  6e-16
 3: -5.9982e-01 -6.0825e-01  8e-03  5e-17  5e-16
 4: -5.9999e-01 -6.0010e-01  1e-04  2e-17  3e-16
 5: -5.9999e-01 -6.0001e-01  2e-05  1e-17  3e-16
 6: -5.9999e-01 -6.0000e-01  3e-06  2e-17  3e-16
 7: -5.9999e-01 -6.0000e-01  5e-07  3e-17  4e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.0033e+01 -1.9239e+00  5e+02  2e+01  1e-15
 1: -9.8792e-01 -1.8697e+00  6e+00  2e-01  1e-15
 2: -5.9123e-01 -1.3063e+00  7e-01  4e-17  6e-16
 3: -5.9982e-01 -6.0825e-01  8e-03  5e-17  5e-16
 4: -5.9999e-01 -6.0010e-01  1e-04  2e-17  3e-16
 5: -5.9999e-01 -6.0001e-01  2e-05  1e-17  3e-16
 6: -5.9999e-01 -6.0000e-01  3e-06  2e-17  3e-16
 7: -5.9999e-01 -6.0000e-01  5e-07  3e-17  4e-16
Optimal solution found.
-0.3258614450421773
-0.32586655345437426
-0.32585477218464987
-0.3258034

     pcost       dcost       gap    pres   dres
 0: -4.1397e+01 -1.8713e+00  4e+02  2e+01  1e-15
 1: -1.0175e+00 -1.8624e+00  5e+00  2e-01  1e-15
 2: -6.2423e-01 -1.3133e+00  7e-01  4e-17  6e-16
 3: -6.3984e-01 -6.5363e-01  1e-02  2e-17  5e-16
 4: -6.4000e-01 -6.4014e-01  1e-04  3e-17  3e-16
 5: -6.4000e-01 -6.4000e-01  2e-06  2e-17  4e-16
 6: -6.4000e-01 -6.4000e-01  2e-07  3e-17  3e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.1397e+01 -1.8713e+00  4e+02  2e+01  1e-15
 1: -1.0175e+00 -1.8624e+00  5e+00  2e-01  1e-15
 2: -6.2423e-01 -1.3133e+00  7e-01  4e-17  6e-16
 3: -6.3984e-01 -6.5363e-01  1e-02  2e-17  5e-16
 4: -6.4000e-01 -6.4014e-01  1e-04  3e-17  3e-16
 5: -6.4000e-01 -6.4000e-01  2e-06  2e-17  4e-16
 6: -6.4000e-01 -6.4000e-01  2e-07  3e-17  3e-16
Optimal solution found.
-0.28089882107900455
-0.28089920805733976
-0.28089866850653916
-0.280899395651333
-0.28089920805733976
-0.28089841624208794
-0.2808984748435573
-0.280898900225774
-0.2808

...

     pcost       dcost       gap    pres   dres
 0: -7.8993e+00 -1.2118e+02  4e+02  2e+00  6e-16
 1: -4.9157e+00 -5.2809e+01  5e+01  3e-02  5e-16
 2: -7.5906e+00 -1.7339e+01  1e+01  6e-03  5e-16
 3: -9.3880e+00 -1.2455e+01  3e+00  1e-03  4e-16
 4: -1.0251e+01 -1.0901e+01  7e-01  2e-04  5e-16
 5: -1.0481e+01 -1.0550e+01  7e-02  1e-05  5e-16
 6: -1.0512e+01 -1.0513e+01  9e-04  1e-07  5e-16
 7: -1.0513e+01 -1.0513e+01  9e-06  1e-09  4e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -7.8993e+00 -1.2118e+02  4e+02  2e+00  6e-16
 1: -4.9157e+00 -5.2809e+01  5e+01  3e-02  5e-16
 2: -7.5906e+00 -1.7339e+01  1e+01  6e-03  5e-16
 3: -9.3880e+00 -1.2455e+01  3e+00  1e-03  4e-16
 4: -1.0251e+01 -1.0901e+01  7e-01  2e-04  5e-16
 5: -1.0481e+01 -1.0550e+01  7e-02  1e-05  5e-16
 6: -1.0512e+01 -1.0513e+01  9e-04  1e-07  5e-16
 7: -1.0513e+01 -1.0513e+01  9e-06  1e-09  4e-16
Optimal solution found.
1.4938771436291647
-1.2819772680561552
0.9800411747389202
1.314764614486

     pcost       dcost       gap    pres   dres
 0: -5.2635e+01 -1.8656e+02  5e+02  2e+00  2e-15
 1: -4.3621e+01 -1.1465e+02  7e+01  1e-15  2e-15
 2: -4.7556e+01 -5.6476e+01  9e+00  1e-15  1e-15
 3: -5.0335e+01 -5.1876e+01  2e+00  3e-16  2e-15
 4: -5.1107e+01 -5.1324e+01  2e-01  1e-15  2e-15
 5: -5.1225e+01 -5.1240e+01  1e-02  2e-15  2e-15
 6: -5.1233e+01 -5.1234e+01  2e-04  3e-15  2e-15
 7: -5.1234e+01 -5.1234e+01  2e-06  2e-16  2e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -5.2635e+01 -1.8656e+02  5e+02  2e+00  2e-15
 1: -4.3621e+01 -1.1465e+02  7e+01  1e-15  2e-15
 2: -4.7556e+01 -5.6476e+01  9e+00  1e-15  1e-15
 3: -5.0335e+01 -5.1876e+01  2e+00  3e-16  2e-15
 4: -5.1107e+01 -5.1324e+01  2e-01  1e-15  2e-15
 5: -5.1225e+01 -5.1240e+01  1e-02  2e-15  2e-15
 6: -5.1233e+01 -5.1234e+01  2e-04  3e-15  2e-15
 7: -5.1234e+01 -5.1234e+01  2e-06  2e-16  2e-15
Optimal solution found.
-0.0870945350336832
-0.5629002185584037
-0.09636037945537648
-0.06865493

     pcost       dcost       gap    pres   dres
 0: -7.6755e+01 -2.0102e+02  6e+02  2e+00  2e-15
 1: -5.3789e+01 -1.2983e+02  8e+01  2e-15  2e-15
 2: -5.5759e+01 -5.7309e+01  2e+00  3e-15  2e-15
 3: -5.5889e+01 -5.6078e+01  2e-01  5e-16  1e-15
 4: -5.5933e+01 -5.5965e+01  3e-02  9e-16  2e-15
 5: -5.5948e+01 -5.5955e+01  7e-03  2e-15  1e-15
 6: -5.5950e+01 -5.5953e+01  3e-03  6e-16  2e-15
 7: -5.5951e+01 -5.5953e+01  1e-03  1e-15  2e-15
 8: -5.5952e+01 -5.5952e+01  5e-05  5e-15  2e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -7.6755e+01 -2.0102e+02  6e+02  2e+00  2e-15
 1: -5.3789e+01 -1.2983e+02  8e+01  2e-15  2e-15
 2: -5.5759e+01 -5.7309e+01  2e+00  3e-15  2e-15
 3: -5.5889e+01 -5.6078e+01  2e-01  5e-16  1e-15
 4: -5.5933e+01 -5.5965e+01  3e-02  9e-16  2e-15
 5: -5.5948e+01 -5.5955e+01  7e-03  2e-15  1e-15
 6: -5.5950e+01 -5.5953e+01  3e-03  6e-16  2e-15
 7: -5.5951e+01 -5.5953e+01  1e-03  1e-15  2e-15
 8: -5.5952e+01 -5.5952e+01  5e-05  5e-15  2e-1

     pcost       dcost       gap    pres   dres
 0: -7.7152e+01 -2.0115e+02  6e+02  2e+00  2e-15
 1: -5.3915e+01 -1.3000e+02  8e+01  6e-16  2e-15
 2: -5.5872e+01 -5.7169e+01  1e+00  3e-16  2e-15
 3: -5.5997e+01 -5.6013e+01  2e-02  2e-15  2e-15
 4: -5.5999e+01 -5.6001e+01  2e-03  3e-15  2e-15
 5: -5.5999e+01 -5.6000e+01  3e-04  3e-15  2e-15
 6: -5.5999e+01 -5.6000e+01  8e-05  5e-15  2e-15
 7: -5.5999e+01 -5.6000e+01  4e-05  9e-16  2e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -7.7152e+01 -2.0115e+02  6e+02  2e+00  2e-15
 1: -5.3915e+01 -1.3000e+02  8e+01  6e-16  2e-15
 2: -5.5872e+01 -5.7169e+01  1e+00  3e-16  2e-15
 3: -5.5997e+01 -5.6013e+01  2e-02  2e-15  2e-15
 4: -5.5999e+01 -5.6001e+01  2e-03  3e-15  2e-15
 5: -5.5999e+01 -5.6000e+01  3e-04  3e-15  2e-15
 6: -5.5999e+01 -5.6000e+01  8e-05  5e-15  2e-15
 7: -5.5999e+01 -5.6000e+01  4e-05  9e-16  2e-15
Optimal solution found.
-0.3777476855341696
-0.3777966155038297
-0.3777484658645923
-0.377745698

KeyboardInterrupt: 

In [49]:
clf.score(xTest, yTest)


AttributeError: 'GridSearchCV' object has no attribute 'scorer_'

In [40]:
clf.best_params_

{'C': 0.1, 'sigma': 0.9}

In [41]:
best_params = clf.best_params_
best_C = best_params['C']
best_sigma = best_params['sigma']
(best_C, best_sigma)

(0.1, 0.9)

In [46]:
if __name__ == "__main__":
    import pylab as pl           
    def hyp_svm():
        
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            typ = 2            
            best_model = HYP_SVM(C=best_C, kernel=gaussian_kernel, sigma=best_sigma)  
            
            best_model.fit(X_train, y_train)
            best_model.score(X_test, y_test)            
            
    hyp_svm()

     pcost       dcost       gap    pres   dres
 0: -7.5487e+00 -2.1155e+01  5e+02  2e+01  7e-16
 1: -3.4718e+00 -1.9578e+01  2e+01  3e-01  7e-16
 2: -3.4729e+00 -5.2535e+00  2e+00  5e-03  4e-16
 3: -3.8630e+00 -4.4904e+00  6e-01  1e-03  4e-16
 4: -3.9958e+00 -4.2781e+00  3e-01  5e-04  4e-16
 5: -4.0560e+00 -4.1769e+00  1e-01  2e-04  4e-16
 6: -4.0940e+00 -4.1157e+00  2e-02  8e-06  4e-16
 7: -4.1026e+00 -4.1047e+00  2e-03  1e-15  4e-16
 8: -4.1035e+00 -4.1036e+00  1e-04  4e-16  4e-16
 9: -4.1036e+00 -4.1036e+00  2e-06  7e-16  4e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -7.5487e+00 -2.1155e+01  5e+02  2e+01  7e-16
 1: -3.4718e+00 -1.9578e+01  2e+01  3e-01  7e-16
 2: -3.4729e+00 -5.2535e+00  2e+00  5e-03  4e-16
 3: -3.8630e+00 -4.4904e+00  6e-01  1e-03  4e-16
 4: -3.9958e+00 -4.2781e+00  3e-01  5e-04  4e-16
 5: -4.0560e+00 -4.1769e+00  1e-01  2e-04  4e-16
 6: -4.0940e+00 -4.1157e+00  2e-02  8e-06  4e-16
 7: -4.1026e+00 -4.1047e+00  2e-03  1e-15  4e-1

In [ ]:
#clf_svm = svm.SVC(kernel='rbf', gamma=0.001, C=100)
#clf_svm.fit(X_train, y_train)
#y_pred_svm = clf_svm.predict(X_test) 
#acc_svm = accuracy_score(y_test, y_pred_svm)
#print ("Overall RBF KERNEL SVM accuracy: ",acc_svm)

In [ ]:
from cvxopt import matrix
class SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def fit(self, X, y):
        self.kernel = gaussian_kernel
        n_samples, n_features = X.shape
        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = gaussian_kernel(X[i], X[j])
               # print(K[i,j])
        print(K.shape)

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
       # print(a)
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
                #print(self.w)
        else:
            self.w = None

    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [ ]:
if __name__ == "__main__":
    import pylab as pl           
    def normal_svm():
        
        clf = SVM(C=100.0)
        
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        for train_index, test_index in skf.split(X, y):
            #print("TRAIN:", train_index, "\nTEST:", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            clf.fit(X_train, y_train)    
            y_predict = clf.predict(X_test)
            gm(y_predict,y_test)
            correct = np.sum(y_predict == y_test)
            mse = mean_squared_error(y_test, y_predict)
            print("%d out of %d predictions correct" % (correct, len(y_predict)))
            print("Accuracy",correct/len(y_predict))
            print("Errore quadratico medio: ", mse)
    normal_svm()   